# Call

`Call` is a simple object for one-off calls to LLMs. It does not track history or token usage.

In [1]:
from IPython.display import Image, display

from irouter import Call
from irouter.base import nb_markdown

# To load OPENROUTER_API_KEY from .env file create a .env file at the root of the project with OPENROUTER_API_KEY=your_api_key
# Alternatively pass api_key=your_api_key to the Call class
from dotenv import load_dotenv

load_dotenv()

True

In this notebook we will use free tiers for Moonshot AI's Kimi K2 and Google's Gemma 3N. 

An overview of all available models can be found by calling `get_all_models`:
```python
from irouter.base import get_all_models
model_slugs = get_all_models()
model_slugs
```

In [2]:
model_names = ["moonshotai/kimi-k2:free", "google/gemma-3n-e2b-it:free"]

test_message = "Who played the guitar solo on Steely Dan's Kid Charlemagne?"

# Single Model

The simplest way to use `Call` is use a single LLM by giving a model slug.

In this example we initialize a `Call` object with the free tier of Moonshot AI's Kimi-K2 LLM.

To set the API key you can either set an environment variable for `OPENROUTER_API_KEY` to your project or pass `api_key` when initializing `Call`.

In [3]:
c = Call(model_names[0])
# or
# c = Call(model_names[0], api_key="your_api_key")

In [4]:
nb_markdown(c(test_message))

The guitar solo on Steely Dan's *"Kid Charlemagne"* was played by **Larry Carlton**.

If needed you can get the raw `ChatCompletion` object from a call by setting `raw=True`.

In [5]:
c(test_message, raw=True)

ChatCompletion(id='gen-1754218891-403swd05iCIF6mQJT5BR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The guitar solo on Steely Dan\'s "Kid Charlemagne" was played by **Larry Carlton**.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1754218891, model='moonshotai/kimi-k2:free', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=22, prompt_tokens=40, total_tokens=62, completion_tokens_details=None, prompt_tokens_details=None), provider='Chutes')

# Multiple Models

Calling multiple models at once is as simple as giving a list of model slugs to `Call`.

In [6]:
mc = Call(model_names)

If multiple models are giving a call will return a dictionary mapping with LLM slug and response for easy parsing.

In [7]:
resps = mc(test_message)
resps

{'moonshotai/kimi-k2:free': "The guitar solo on Steely Dan's *Kid Charlemagne* was played by **Larry Carlton**.",
 'google/gemma-3n-e2b-it:free': 'The guitar solo on Steely Dan\'s "Kid Charlemagne" was played by **Steve Gadd**. \n\nThis is a bit of a trick question, as Steve Gadd is primarily known as a drummer! He was actually a session musician who played a distinctive, melodic guitar solo on the song, which was recorded using a combination of real guitar and synthesized sounds. It\'s a very unusual and memorable solo for someone not traditionally known for playing guitar.\n'}

It can then be parsed.

In [8]:
nb_markdown(resps["google/gemma-3n-e2b-it:free"])

The guitar solo on Steely Dan's "Kid Charlemagne" was played by **Steve Gadd**. 

This is a bit of a trick question, as Steve Gadd is primarily known as a drummer! He was actually a session musician who played a distinctive, melodic guitar solo on the song, which was recorded using a combination of real guitar and synthesized sounds. It's a very unusual and memorable solo for someone not traditionally known for playing guitar.


# Image Input

`Call` supports passing image URLs or local image paths. Make sure to use a model that supports image input like `gpt-4o-mini`. Sometimes image input models require a system prompt to be passed. The default system prompt for `Call` is empty so you may need to pass a system prompt.

In [9]:
image_url = "https://www.petlandflorida.com/wp-content/uploads/2022/04/shutterstock_1290320698-1-scaled.jpg"
display(Image(url=image_url, width=300))

## Image URL

In [10]:
ic = Call("gpt-4o-mini", system="You analyze images.")
ic(
    [
        "https://www.petlandflorida.com/wp-content/uploads/2022/04/shutterstock_1290320698-1-scaled.jpg",
        "What is in the image?",
    ]
)

'The image features a cute corgi puppy sitting down. The puppy has a mix of tan and white fur, with one ear perked up. The background is a soft green blur, suggesting an outdoor setting.'

## Local Image Path

Here the same image is passed as a local image path.

In [11]:
ic(["../assets/puppy.jpg", "What is in the image?"])

'The image shows an adorable puppy, likely a Corgi, with a light tan and white coat. The puppy has a sitting posture, one ear upright, and a soft expression. The background appears to be blurred greenery, suggesting an outdoor setting.'

# PDF Input

`irouter` handles PDF URL and local PDF file paths out of the box. 

PDFs are processed as follows:
- If the PDF is a local file, it will be processed as `base64`.
- If a PDF URL is given and the selected LLM supports native file processing that is the default, which is charged as input tokens.
- If a PDF URL is given and the selected LLM does not support native file processing, the `mistral-ocr` engine is used to read the PDF, which costs $2 per 1,000 pages.

In [12]:
pdf_path = "https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf"

c([pdf_path, "What is introduced in this paper?"])

'The paper introduces the **Transformer**, a new neural network architecture for sequence transduction (e.g., machine translation). Distinctively, it **relies entirely on attention mechanisms**, specifically **multi-head self-attention**, and **eliminates the use of recurrence and convolutions** entirely. The model achieves state-of-the-art results on translation tasks (WMT 2014 English-to-German and English-to-French) while being more parallelizable and computationally efficient.'

You can set custom PDF parsers through the `extra_body parameter`. For example if you want free PDF parsing, specify the `pdf-text` engine in the `extra_body` parameter. Check out [this docs page](https://openrouter.ai/docs/features/images-and-pdfs#plugin-configuration) to learn more about the format of `plugins`.

In [14]:
extra_body = {"plugins": [{"id": "file-parser", "pdf": {"engine": "pdf-text"}}]}
c([pdf_path, "What is introduced in this paper?"], extra_body=extra_body)

'The paper introduces the **Transformer**, a **new neural network architecture** for sequence transduction tasks that **relies entirely on attention mechanisms**, eliminating the need for recurrence or convolutions. Key innovations include:\n\n1. **Self-attention (Multi-Head Attention)** – A mechanism that allows the model to relate positions within a sequence without sequential computation.  \n2. **Encoder-decoder stacks** – Composed of layers with **multi-head self-attention** and **position-wise feed-forward networks**, coupled with residual connections and layer normalization.  \n3. **Positional encodings** – Sinusoidal position embeddings to retain sequence order since recurrence is removed.  \n4. **Scalable training** – Faster convergence compared to RNN/CNN-based models (e.g., 38.1 BLEU on EN-DE in 12 hours on 8 GPUs).  \n\nThe Transformer **departs from recurrent and convolutional layers**, enabling **parallel computation** and **better performance** on machine translation task

# Messages 

For certain cases you might want to pass a list of messages to the Call class. For example to simulate a conversation.

In that case messages should be defined as a **list of dicts which define `role` and `content`**.

NOTE: The `system` parameter will be ignored if you pass a list of messages. Therefore when using this approach optionally also define a system message in your list of messages.

Here is an example of a conversation between a user and an assistant with a system message at the start.

In [15]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Amsterdam."},
    {
        "role": "user",
        "content": "No silly, why would you say Amsterdam is the capital of France?",
    },
]

c = Call(model_names[0])
c(messages)

"You're absolutely right—my mistake! The capital of France, of course, is **Paris**."

I hope this gives you a good overview of how to work with `Call`. If you would like to track history and token usage check out the `Chat` object in `irouter`.